In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import cv2
from src.lab1.lab1_challenge2 import *
from src.lab1.info_image import *
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from src.lab3.lab3 import *
from tensorflow.train import Saver

SIZE = 7

In [3]:
data = get_all_masks(10)
X_train = []; y_train = []
test_data = 0
# On crée les jeux d'entrainements....
for img_name, mask in data:
    img = cv2.imread(img_name, 0)
    for (roi, center, w, h, angle) in sliding_windows(mask, SIZE, SIZE):
        x = extract_batch(img, center, SIZE, SIZE)
        y = extract_mask(mask, center, SIZE, SIZE)
        if x is not None and y is not None:
            X_train.append(x)
            y_train.append(y)
            test_data += 1
print(y_train)
print("Nombre d'imagette : ", test_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Déclaration du modele fully conected

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(SIZE, SIZE)))
model.add(Dense(98, activation = "relu"))
model.add(Dense(98, activation="relu"))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 49)                0         
_________________________________________________________________
dense (Dense)                (None, 98)                4900      
_________________________________________________________________
dense_1 (Dense)              (None, 98)                9702      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 198       
Total params: 14,800
Trainable params: 14,800
Non-trainable params: 0
_________________________________________________________________


### Entrainement

In [ ]:
X_train = np.array(X_train)
y_train = y_train
history = model.fit(X_train,
                    y_train, epochs = 5)

### Test

In [ ]:
print(SIZE)
test_data = get_test_masks()[:3]
for img_name, mask in test_data:
    list_good = []
    list_to_predict = []
    img = cv2.imread(img_name,0)
    x_size = mask.shape[0]//SIZE
    y_size = mask.shape[1]//SIZE
    for (roi, center, w, h, angle) in sliding_windows(mask, SIZE, SIZE):
        d = extract_batch(img, center, SIZE, SIZE)
        m = extract_mask(mask, center, SIZE, SIZE)
        if d is not None and m is not None:
            list_good.append(m)
            list_to_predict.append(d)

    print("test batches", len(list_to_predict))
    prediction = model.predict(np.array(list_to_predict))
    print("prediction shape", prediction.shape)
    print("image", img_name)

    img_reconstruct = np.zeros(img.shape)

    for x in range(x_size):
        for y in range(y_size):
            l = x * x_size + y
            print(prediction[l])
            if prediction[l][1] > 0.5:
                img_reconstruct[x*SIZE:(x+1)*SIZE, y*SIZE:(y+1)*SIZE] = img[x*SIZE:(x+1)*SIZE, y*SIZE:(y+1)*SIZE]

    print("saved image", img_name[0] + "test_model.png")
    cv2.imwrite(img_name[0] + "test_model.png", img_reconstruct)